In [0]:
#Parametro de carga 

dbutils.widgets.text("param_ano", "", "Ano Proposicoes")

In [0]:

from datetime import datetime
from utils.api import buscar_dados_paginado
from pyspark.sql.functions import current_timestamp

In [0]:
# Informações Unity Catalog

catalogo =  "fiscaliza_legislativo"
schema = "bronze"
tabela = "proposicoes"

TABELA = f"{catalogo}.{schema}.{tabela}"

In [0]:
#Dados API

param_ano = dbutils.widgets.get("param_ano")
ano =  datetime.now().year if param_ano == "" else param_ano


API = 'https://dadosabertos.camara.leg.br/api/v2'
ENDPOINT = '/proposicoes'
params = {
    "ano" : ano
}
BaseURL = f"{API}{ENDPOINT}"

# Inicia execução

In [0]:
TInicio = datetime.now()
print(TInicio)

In [0]:
# Executa API

dados = buscar_dados_paginado(base_url = BaseURL, params = params, itensporPagina=100)

In [0]:
#Transforma dados em DataFrame

df_proposicoes = spark.createDataFrame(dados)

In [0]:
# Adicona Data de Ingestão 

df_proposicoes = df_proposicoes\
                    .withColumn("_data_ingestao", current_timestamp())  

In [0]:
# Cria Tabela Delta 

df_proposicoes.write.format("delta")\
                  .mode("overwrite")\
                  .option("mergeSchema", "true")\
                  .option("replaceWhere", f"ano = {ano}")\
                  .saveAsTable(TABELA)

In [0]:
TFim = datetime.now()
print(TFim)
print("Duração:", TFim - TInicio)